### Train-on-all-but-one, decode-on-one, experiment #2

Generate decoded sample files for input to e_ F1 classifiers  
Read from normalized-within-cohort, 5k common mean absolute deviation (MAD) data file set

Demo version, created 2022-11-17,  
from b_train-on-one.ipynb

In [1]:
%whos

Interactive namespace is empty.


### Imports

In [2]:
import time

import glob as glob

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization, Activation, Layer
from tensorflow.keras import metrics, optimizers
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# import umap.umap_ as umap - to d_

In [5]:
# target: g_devel/train-on-all_decoded, then read into the multi-experiment e_RF_DT_org-dec.ipynb

# should be renamed e_RF_DT_vae_inp-dec.ipynb

In [7]:
plt_v=17 # Train-on-all, experiment 4, using normalize within cohort data: g_devel/data_GEXP_5k_col_norm

### Build Model

In [4]:
# Function set for leave one out loop
def compute_latent(x): # x:
    mu, sigma = x
    batch = K.shape(mu)[0]
    dim = K.shape(mu)[1]
    eps = K.random_normal(shape=(batch,dim), mean=0., stddev=1.0 )
    return mu + K.exp(sigma/2)*eps

In [5]:
# Set custom layer , will need all this at start of shell script
class CustomVariationalLayer(Layer):
    """
    Define a custom layer
    """
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

In [6]:
class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa

    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)

In [ ]:
# guess we need to restack the frames and hold one out
# start with getting a cohort list

In [12]:
nrm_wthn_pths

['data_GEXP_5k_col_norm/ACC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/BLCA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/BRCA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/CESC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/COADREAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/ESCC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/GEA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/HNSC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/KIRCKICH_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/KIRP_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LIHCCHOL_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LUAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LUSC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/MESO_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/OV_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PAAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PCPG_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PRAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/SARC_gexp_col_norm_5k.tsv',
 'da

In [14]:
len(nrm_wthn_pths)

25

In [22]:
full_stack = pd.DataFrame() # Production cell 1, stack up the cancers
for path in nrm_wthn_pths:
    print(path)
    train_df = pd.read_csv(path, sep="\t", index_col=0)
    full_stack = pd.concat( [full_stack, train_df], axis = 0 )
    # break

data_GEXP_5k_col_norm/ACC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/BLCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/BRCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/CESC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/COADREAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/ESCC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/GEA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/HNSC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/KIRCKICH_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/KIRP_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LIHCCHOL_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LUAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LUSC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/MESO_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/OV_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PAAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PCPG_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PRAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/SARC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/SKCM_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/TGCT_gexp_

In [23]:
full_stack

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-OR-A5J1,ACC_2,0.000182,0.360981,0.001892,0.007364,0.000000,0.000000,0.000000,0.000284,0.006090,...,0.246175,0.542221,0.402793,0.335923,0.241655,0.232638,0.137530,0.278195,0.306376,0.078349
TCGA-OR-A5J2,ACC_2,0.000662,0.028076,0.016630,0.094906,0.001782,0.002093,0.000000,0.052871,0.002012,...,0.057344,0.810534,0.198127,0.339276,1.000000,0.255464,0.184041,0.413199,0.303635,0.039638
TCGA-OR-A5J3,ACC_3,0.000675,0.504978,0.004538,0.036593,0.002623,0.001027,0.000000,0.002378,0.000851,...,0.457210,0.488562,0.499732,0.410338,0.493998,0.576924,0.231120,0.519779,0.540690,0.028952
TCGA-OR-A5J5,ACC_3,0.039602,0.519118,0.005965,0.081847,0.000000,0.001342,0.000000,0.000142,0.001235,...,0.145073,0.509690,0.383014,0.394384,0.254431,0.305898,0.222402,0.371465,0.304460,0.073477
TCGA-OR-A5J6,ACC_1,0.000141,0.017320,0.001465,0.003721,0.000000,0.000000,0.000000,0.007885,0.002986,...,0.083668,0.411400,0.346296,0.393668,0.321142,0.231314,0.780143,0.318338,0.992842,0.742167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WC-AA9E,UVM_3,0.030804,0.005387,0.003946,0.020549,0.102395,0.082625,0.076004,0.019509,0.055003,...,0.393935,0.610252,0.344880,0.686383,0.853623,0.598790,0.528979,0.337590,0.276482,0.037362
TCGA-YZ-A980,UVM_1,0.028575,0.077464,0.094785,0.145219,0.004130,0.000684,0.001621,0.000118,0.313738,...,0.448846,0.425112,0.702758,0.458945,0.515307,0.449282,0.272083,0.430847,0.303070,0.049678
TCGA-YZ-A982,UVM_2,0.000000,0.005380,0.007426,0.031989,0.023712,0.011788,0.000000,0.000149,0.295075,...,0.551889,0.250576,0.607254,0.601735,0.518152,0.364811,0.395582,0.302280,0.306822,0.009950
TCGA-YZ-A983,UVM_3,0.021407,0.475421,0.025232,0.039406,0.179425,0.042036,0.159049,0.000309,0.082759,...,0.341519,0.404413,0.578043,0.483949,0.766207,0.827500,0.300552,0.386852,0.201858,0.024924


In [24]:
full_stack.Labels.str.split('_').str[0].unique()

array(['ACC', 'BLCA', 'BRCA', 'CESC', 'COADREAD', 'ESCC', 'GEA', 'HNSC',
       'KIRCKICH', 'KIRP', 'LIHCCHOL', 'LUAD', 'LUSC', 'MESO', 'OV',
       'PAAD', 'PCPG', 'PRAD', 'SARC', 'SKCM', 'TGCT', 'THCA', 'THYM',
       'UCEC', 'UVM'], dtype=object)

In [27]:
c_lst = full_stack.Labels.str.split('_').str[0].unique() # Production cell 2

In [25]:
full_stack.Labels.str.contains('ACC') # gives the boolean

TCGA-OR-A5J1     True
TCGA-OR-A5J2     True
TCGA-OR-A5J3     True
TCGA-OR-A5J5     True
TCGA-OR-A5J6     True
                ...  
TCGA-WC-AA9E    False
TCGA-YZ-A980    False
TCGA-YZ-A982    False
TCGA-YZ-A983    False
TCGA-YZ-A985    False
Name: Labels, Length: 8009, dtype: bool

In [26]:
~full_stack.Labels.str.contains('ACC') # tilda is friend

TCGA-OR-A5J1    False
TCGA-OR-A5J2    False
TCGA-OR-A5J3    False
TCGA-OR-A5J5    False
TCGA-OR-A5J6    False
                ...  
TCGA-WC-AA9E     True
TCGA-YZ-A980     True
TCGA-YZ-A982     True
TCGA-YZ-A983     True
TCGA-YZ-A985     True
Name: Labels, Length: 8009, dtype: bool

In [28]:
for c in c_lst:
    stack_less_one = full_stack[~full_stack.Labels.str.contains(c)]
    break

In [30]:
8009-7933 # sample count for adrenocortical, check

76

In [33]:
# Train-on-one using within-cohort normed data
# for c in stacked.Labels.str.split('_').str[0].unique():
c_lst = full_stack.Labels.str.split('_').str[0].unique()

plt_v = 17
nrm_wthn_pths = sorted(glob.glob('data_GEXP_5k_col_norm/*.tsv'))   
for c in c_lst:
    stack_less_one = full_stack[~full_stack.Labels.str.contains(c)]
    decode_df = full_stack[full_stack.Labels.str.contains(c)]
    # break
    features = stack_less_one.columns[1:]
    
    original_dim = len(features)
    feature_dim = len(features)
    latent_dim = 100
    
    encoder_inputs = keras.Input(shape=(feature_dim,))
    z_mean_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_1")(encoder_inputs)
    z_mean_dense_batchnorm = layers.BatchNormalization()(z_mean_dense_linear)
    z_mean_encoded = layers.Activation('relu')(z_mean_dense_batchnorm)

    z_log_var_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_2")(encoder_inputs)
    z_log_var_dense_batchnorm = layers.BatchNormalization()(z_log_var_dense_linear)
    z_log_var_encoded = layers.Activation('relu')(z_log_var_dense_batchnorm)

    latent_space = layers.Lambda(compute_latent, output_shape=(latent_dim,), name="latent_space")([z_mean_encoded, z_log_var_encoded])

    decoder_to_reconstruct = layers.Dense(feature_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
    decoder_outputs = decoder_to_reconstruct(latent_space)

    learning_rate = 0.0005
    kappa = 1

    beta = K.variable(0)

    adam = optimizers.Adam(learning_rate=learning_rate)
    vae_layer = CustomVariationalLayer()([encoder_inputs, decoder_outputs])
    vae = Model(encoder_inputs, vae_layer)
    vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

    epochs=100

    fit_start = time.time()
    history = vae.fit(stack_less_one.iloc[:, 1:],  #       <-------- fit the VAE here
                epochs=epochs, batch_size=50, shuffle=True,
                callbacks=[WarmUpCallback(beta, kappa)],
                     # verbose=0 #  <---------------------- turn this on for production
                     )
    fit_end = time.time() - fit_start

    plt.plot(history.history['loss'],label="loss")

    plt.title(
        c+' held out from fitting'+
        '\nTybalt VAE train\n'
             )
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.annotate('Feature set = '+'common MAD 5000k'+
                 '\nLatent dim = '+str(latent_dim)+
                 '\nLayer type = dense'+
                 '\nNormalization = divide by max(),\n'+
                 '                       column-wise,\n'+
                 '                       within each cohort'+
                 '\nFit time = '+str(fit_end),
                xy=(.4, .8), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                )

    plt.legend(loc="lower left")
    plt.savefig(
        'train-on-all_loss_plots/'+c+'_held_out_'+
        '_fit_on_n=24_cohorts_'+str(plt_v)+'.png', bbox_inches='tight') # version 17, norm within cohort
    plt.close()
        
    encoder = Model(encoder_inputs, z_mean_encoded)
    decoder_input = keras.Input(shape=(latent_dim, ))
    _x_decoded_mean = decoder_to_reconstruct(decoder_input)
    decoder = Model(decoder_input, _x_decoded_mean)

    y_df = decode_df.Labels

    decoded = pd.DataFrame(decoder.predict(encoder.predict(decode_df.iloc[:, 1:])),
                           index=decode_df.index, columns=decode_df.iloc[:, 1:].columns)
    
    latent_object = pd.DataFrame(encoder.predict(decode_df.iloc[:, 1:]),
                 index=decode_df.index,
                    # columns=decode_df.iloc[:, 1:].columns # Columns are different because this is latent space
            )
    
    latent_object.to_csv('train-on-all_latent_objects/
    break
    decoded_labeled = pd.concat( [pd.DataFrame(y_df), decoded] , axis =1)
    decoded_labeled.to_csv(
        'train-on-all_decoded/trn-on-n24_'+c+'_held_out'+
        str(round(fit_end,2))+'_fit_time.tsv',
        sep = '\t')

    print(str(c) + ' done')
    print(' ')

print('Train-on-all experiment done')

Instructions for updating:
Colocations handled automatically by placer.
Metal device set to: Apple M1 Max
Train on 7933 samples
Epoch 1/100


2022-11-17 12:46:24.919535: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-17 12:46:24.919688: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-17 12:46:24.923671: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-17 12:46:24.927269: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-17 12:46:24.927818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 12:46:24.940699: I tensorflow/core/grappler/optimizers/custom_graph

7933/7933 [==============================] - 3s 326us/sample - loss: 2706.6794
Epoch 2/100
 650/7933 [=>............................] - ETA: 1s - loss: 2452.0567

2022-11-17 12:46:27.681705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7933/7933 [==============================] - 2s 260us/sample - loss: 2421.1944
Epoch 3/100
7933/7933 [==============================] - 2s 263us/sample - loss: 2377.0757
Epoch 4/100
7933/7933 [==============================] - 2s 258us/sample - loss: 2349.3103
Epoch 5/100
7933/7933 [==============================] - 2s 261us/sample - loss: 2330.4585
Epoch 6/100
7933/7933 [==============================] - 2s 264us/sample - loss: 2318.4606
Epoch 7/100
7933/7933 [==============================] - 2s 272us/sample - loss: 2309.4576
Epoch 8/100
7933/7933 [==============================] - 2s 264us/sample - loss: 2302.7106
Epoch 9/100
7933/7933 [==============================] - 2s 261us/sample - loss: 2297.6158
Epoch 10/100
7933/7933 [==============================] - 2s 262us/sample - loss: 2293.7243
Epoch 11/100
7933/7933 [==============================] - 2s 262us/sample - loss: 2289.7578
Epoch 12/100
7933/7933 [==============================] - 2s 272us/sample - loss: 2287.0808
Epoch 13

/Users/karlberb/miniforge3/envs/tf_M1_py3dot9/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-17 12:49:57.696038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-17 12:49:57.787728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


NameError: name 'decode_df_df' is not defined

In [39]:
pd.DataFrame(encoder.predict(decode_df.iloc[:, 1:]),
                 index=decode_df.index,
                    # columns=decode_df.iloc[:, 1:].columns # Columns are different because this is latent space
            )

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
TCGA-OR-A5J1,0.919229,0.536419,2.750913,0.015861,0.000000,1.544685,0.334823,0.000000,1.514472,2.803095,...,0.244509,1.566356,3.653692,0.879896,0.000000,0.000000,0.707595,2.042920,0.674109,1.025202
TCGA-OR-A5J2,1.903800,2.250574,0.107363,0.000000,0.000000,2.902470,0.708469,0.000000,0.332290,1.974677,...,0.000000,2.380002,2.906023,0.744098,0.000000,0.000000,0.398547,0.000000,0.000000,0.799846
TCGA-OR-A5J3,1.103231,0.000000,0.070782,0.000000,0.000000,1.492993,0.594472,0.000000,0.000000,0.000000,...,0.000000,3.485821,2.132439,0.060449,1.892141,4.532578,1.913464,1.314799,0.721028,0.719032
TCGA-OR-A5J5,0.000000,0.000000,2.797881,0.000000,0.000000,4.107993,0.000000,3.080622,0.713190,2.032344,...,0.470202,1.470344,1.838027,0.000000,2.618226,3.727907,0.790332,2.207822,1.069085,1.748828
TCGA-OR-A5J6,1.719181,2.722994,1.886564,0.000000,0.590122,0.195153,0.441957,0.076789,2.134021,0.380131,...,1.622998,1.263959,0.000000,0.000000,2.718529,0.851742,0.000000,0.647611,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OU-A5PI,0.321543,1.198746,0.907563,0.000000,0.786951,2.268306,2.018335,0.254274,0.000000,0.000000,...,0.000000,1.386065,0.000000,0.000000,0.000000,2.402194,2.598593,1.745641,0.000000,1.718378
TCGA-PA-A5YG,0.000000,0.878225,2.182189,0.363155,1.121043,1.372840,0.964149,0.000000,0.000000,1.265577,...,0.000000,1.846793,0.535333,0.000000,2.837408,0.760872,0.310979,0.262465,0.320479,1.446502
TCGA-PK-A5H9,0.125829,0.798127,0.000000,0.000000,1.433990,0.822006,1.341544,0.000000,0.000000,1.464653,...,1.726944,0.299907,0.668441,0.720057,1.828589,2.582231,0.000000,1.108835,0.000000,0.391239
TCGA-PK-A5HA,0.534196,3.576379,0.356900,0.273426,0.000000,3.028434,0.000000,0.000000,0.069610,0.000000,...,0.000000,0.297705,1.489907,0.000000,2.574702,2.392457,0.383850,1.108937,0.000000,0.732654


In [34]:
decoded = pd.DataFrame(
    decoder.predict(
        encoder.predict(
            decode_df.iloc[:, 1:])),
                 index=decode_df.index,
                    columns=decode_df.iloc[:, 1:].columns)

In [35]:
decoded

,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,N:GEXP::COL3A1:1281:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-OR-A5J1,0.054576,0.023827,0.010406,0.035705,0.004171,0.003884,0.000932,0.015283,0.028934,0.011016,...,0.531818,0.237136,0.491738,0.479993,0.238765,0.362315,0.242559,0.392529,0.306486,0.135557
TCGA-OR-A5J2,0.001724,0.062549,0.022209,0.153195,0.018499,0.010730,0.011101,0.014621,0.014984,0.046757,...,0.171874,0.422092,0.342351,0.322063,0.687632,0.439612,0.142818,0.421893,0.332992,0.369625
TCGA-OR-A5J3,0.029703,0.106899,0.113270,0.199955,0.092553,0.115291,0.159543,0.063748,0.272052,0.140253,...,0.420611,0.286175,0.469779,0.483301,0.541352,0.603218,0.263958,0.570804,0.434045,0.120655
TCGA-OR-A5J5,0.072395,0.217011,0.122650,0.053369,0.048956,0.032023,0.013507,0.053490,0.042815,0.050577,...,0.563094,0.362338,0.498003,0.338975,0.286529,0.203484,0.314300,0.533343,0.539443,0.151342
TCGA-OR-A5J6,0.130504,0.283170,0.008133,0.038660,0.012690,0.008090,0.004281,0.049964,0.173198,0.015124,...,0.263381,0.299471,0.293953,0.366442,0.374203,0.380202,0.674449,0.402057,0.435811,0.460000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OU-A5PI,0.006202,0.173750,0.044813,0.095060,0.028860,0.022218,0.015385,0.025861,0.039063,0.081801,...,0.270213,0.270423,0.545821,0.506364,0.559265,0.390217,0.256997,0.301979,0.555413,0.114157
TCGA-PA-A5YG,0.017122,0.266059,0.023306,0.125873,0.032808,0.013592,0.010435,0.056975,0.069928,0.055261,...,0.182366,0.338817,0.582576,0.394186,0.519992,0.441457,0.337698,0.400656,0.485707,0.379208
TCGA-PK-A5H9,0.580344,0.179786,0.036926,0.222009,0.013033,0.011089,0.007830,0.081371,0.178893,0.081383,...,0.231996,0.495417,0.327233,0.520180,0.593420,0.309860,0.404554,0.243410,0.202920,0.351568
TCGA-PK-A5HA,0.002548,0.167426,0.093073,0.196195,0.007423,0.013359,0.006516,0.035152,0.044817,0.101840,...,0.147781,0.361958,0.345204,0.397979,0.422134,0.339987,0.203330,0.340868,0.467179,0.302918


In [9]:
c

'ACC'

In [10]:
train_df

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
ACC,,,,,,,,,,,,,,,,,,,,,
TCGA-OR-A5J1,ACC_2,0.000182,0.360981,0.001892,0.007364,0.000000,0.000000,0.0,0.000284,0.006090,...,0.246175,0.542221,0.402793,0.335923,0.241655,0.232638,0.137530,0.278195,0.306376,0.078349
TCGA-OR-A5J2,ACC_2,0.000662,0.028076,0.016630,0.094906,0.001782,0.002093,0.0,0.052871,0.002012,...,0.057344,0.810534,0.198127,0.339276,1.000000,0.255464,0.184041,0.413199,0.303635,0.039638
TCGA-OR-A5J3,ACC_3,0.000675,0.504978,0.004538,0.036593,0.002623,0.001027,0.0,0.002378,0.000851,...,0.457210,0.488562,0.499732,0.410338,0.493998,0.576924,0.231120,0.519779,0.540690,0.028952
TCGA-OR-A5J5,ACC_3,0.039602,0.519118,0.005965,0.081847,0.000000,0.001342,0.0,0.000142,0.001235,...,0.145073,0.509690,0.383014,0.394384,0.254431,0.305898,0.222402,0.371465,0.304460,0.073477
TCGA-OR-A5J6,ACC_1,0.000141,0.017320,0.001465,0.003721,0.000000,0.000000,0.0,0.007885,0.002986,...,0.083668,0.411400,0.346296,0.393668,0.321142,0.231314,0.780143,0.318338,0.992842,0.742167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OU-A5PI,ACC_3,0.001371,0.008835,0.066020,0.021505,0.000000,0.000000,0.0,0.000697,0.002046,...,0.342442,0.369873,0.377868,0.418249,0.551750,0.355757,0.124816,0.525230,0.515990,0.025401
TCGA-PA-A5YG,ACC_1,0.063596,0.007164,0.010554,0.031371,0.001023,0.002404,0.0,0.005909,0.006182,...,0.041263,0.615558,0.462613,0.325033,0.593977,0.227852,0.408315,0.638969,0.301985,0.038206
TCGA-PK-A5H9,ACC_1,0.048619,0.009463,0.004498,0.020371,0.000000,0.000605,0.0,0.001266,0.311465,...,0.045505,0.910912,0.386055,0.313144,0.436139,0.292534,0.428219,0.188895,0.312588,0.134155


In [ ]:
# devel notes

In [31]:
path = 'train-on-one_decoded/ACC_trained_BLCA_decoded_3.44_fit_time.tsv'

In [32]:
check_decoded = pd.read_csv(path, sep="\t", index_col=0)

In [33]:
check_decoded

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
BLCA,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO,BLCA_2,0.258189,0.306483,0.256686,0.261900,0.277183,0.268235,0.255337,0.266503,0.263274,...,0.349611,0.567046,0.458851,0.494862,0.515703,0.451185,0.387592,0.440396,0.466571,0.275499
TCGA-2F-A9KP,BLCA_2,0.241868,0.304308,0.253848,0.255117,0.255665,0.261329,0.231509,0.249237,0.250347,...,0.394670,0.567546,0.441060,0.482957,0.511102,0.473189,0.393672,0.449845,0.470219,0.279433
TCGA-2F-A9KQ,BLCA_2,0.286223,0.350335,0.289034,0.293385,0.304254,0.295316,0.267309,0.287533,0.299055,...,0.371494,0.537936,0.480928,0.493736,0.498572,0.467144,0.430875,0.480783,0.463623,0.321705
TCGA-2F-A9KR,BLCA_1,0.297007,0.353332,0.289260,0.296904,0.307032,0.296238,0.284183,0.284210,0.296633,...,0.401326,0.532719,0.464590,0.481413,0.500898,0.463070,0.425125,0.472673,0.455562,0.309588
TCGA-2F-A9KT,BLCA_1,0.262424,0.341791,0.289164,0.278195,0.277713,0.272221,0.265381,0.277765,0.291722,...,0.389331,0.534316,0.480804,0.482108,0.496206,0.449193,0.400196,0.488183,0.447377,0.305633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,BLCA_1,0.290097,0.359190,0.301596,0.307196,0.321422,0.307218,0.284805,0.301880,0.309024,...,0.401103,0.554085,0.463009,0.520183,0.503187,0.489002,0.421256,0.462873,0.467238,0.328532
TCGA-ZF-AA58,BLCA_1,0.243298,0.295148,0.263807,0.280100,0.305568,0.255755,0.252489,0.279865,0.283731,...,0.368548,0.564527,0.464605,0.528046,0.462646,0.437244,0.383401,0.467405,0.445208,0.314400
TCGA-ZF-AA5H,BLCA_1,0.255045,0.304621,0.293100,0.283789,0.293432,0.258363,0.262900,0.284991,0.294752,...,0.358123,0.577084,0.462294,0.516250,0.502127,0.444514,0.380759,0.487002,0.446064,0.315363
